In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import chardet
import re
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Concatenate, Input, RepeatVector
from tensorflow.keras.preprocessing.text import Tokenizer
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore", message="The structure of `inputs` doesn't match the expected structure")
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_exploded = pd.read_csv('data_output/df_exploded_full.csv')

In [3]:
df_exploded.shape

(1679050, 9)

# Modelo de prediccion de proximo golpe: Plus Otros golpes
Similar al modelo de "Predict_next_player_shot_tuning.ipynb" pero con la inclusion de otros golpes que no sean forehand, backhand o slice, en la categoria "other".

In [4]:
#Conditional column: where the fourth item in the 'Sequence' column contains 's' new column return 's', when 'b' return 'b', when 'f' return 'f', and when other return 'o'

df_exploded_test = df_exploded[df_exploded['Sequence'].str.split().apply(len) > 3]
df_exploded_test['cuarto'] = df_exploded_test['Sequence'].str.split().str[3].apply(
    lambda x: x[0] if isinstance(x, str) and x[0] in ['s', 'b', 'f'] else 'o'
)
df_exploded_test['cuarto'].value_counts(normalize=True)

C:\Users\agusd\AppData\Local\Temp\ipykernel_27324\1369187152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_test['cuarto'] = df_exploded_test['Sequence'].str.split().str[3].apply(


f    0.439085
b    0.344230
o    0.119495
s    0.097190
Name: cuarto, dtype: float64

In [5]:
jugador_elegido = "Roger Federer"

df_exploded = df_exploded.dropna(subset=['Sequence'])

df_exploded_filtered = df_exploded[df_exploded['Sequence'].str.split().apply(len) > 3]
# Filter rows where the fourth item in the 'Sequence' column contains 's', 'b', or 'f'
"""df_exploded_filtered = df_exploded_filtered[
    df_exploded_filtered['Sequence'].str.split().str[3].str.contains(r'^[sbf]')
]"""

print(f"Total de filas: {df_exploded_filtered.shape[0]}")

Total de filas: 297519


In [6]:
df_exploded_filtered

,Player,Sequence,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts
3,Novak Djokovic,6 f1 s1 f2,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
4,Ben Shelton,b19 b2 f3 j2*,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,2,0,0,0.0,0.0,15-0
23,Novak Djokovic,4 b2 b1 b3 f2 f1,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,11,0,0,1.0,1.0,0-0
24,Ben Shelton,b28 f2 s3 f1 b1 b3n@,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,11,0,0,1.0,1.0,0-0
46,Novak Djokovic,6 f3 f1 v3*,20230908-M-US_Open-SF-Novak_Djokovic_-Ben_Shelton,22,0,0,2.0,2.0,15-0
...,...,...,...,...,...,...,...,...,...
1679023,Nicola Pietrangeli,b28 m2 s2 b3 f3 s3 s1,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,164,1,1,5.0,4.0,15-15
1679025,Luis Ayala,4 f3 o1 z3 o1*,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,165,1,1,5.0,4.0,15-30
1679026,Nicola Pietrangeli,s28 m2 f3 m2,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,165,1,1,5.0,4.0,15-30
1679031,Luis Ayala,5 s3 s3 r3,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,167,1,1,5.0,4.0,40-30


In [8]:
df_exploded_filtered['X'] = df_exploded_filtered['Sequence'].str.split().str[:3]
df_exploded_filtered['y'] = df_exploded_filtered['Sequence'].str.split().str[3].str[0]

df_exploded_filtered['y'].value_counts(normalize=True)

C:\Users\agusd\AppData\Local\Temp\ipykernel_27324\3193206234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_filtered['X'] = df_exploded_filtered['Sequence'].str.split().str[:3]
C:\Users\agusd\AppData\Local\Temp\ipykernel_27324\3193206234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exploded_filtered['y'] = df_exploded_filtered['Sequence'].str.split().str[3].str[0]


f    0.439085
b    0.344230
s    0.097190
z    0.022977
r    0.021827
v    0.021726
o    0.015707
m    0.010312
y    0.007156
l    0.006158
u    0.005620
h    0.002010
i    0.001751
p    0.001556
j    0.001425
t    0.000840
k    0.000329
q    0.000081
n    0.000010
d    0.000003
w    0.000003
x    0.000003
Name: y, dtype: float64

In [16]:
#resample df_exploded_filtered so that f = 40% and s = 30% and b = 30%
from sklearn.utils import resample

# Separar las clases
f_class = df_exploded_filtered[df_exploded_filtered['y'] == 'f']
b_class = df_exploded_filtered[df_exploded_filtered['y'] == 'b']
s_class = df_exploded_filtered[df_exploded_filtered['y'] == 's']
o_class = df_exploded_filtered[~df_exploded_filtered['y'].isin(['f', 'b', 's'])]
o_class['y'] = 'o'

# Número total de muestras objetivo
total_samples = len(df_exploded_filtered)

# Calcular el número de muestras para cada clase según las proporciones deseadas
f_target = int(total_samples * 0.25)
b_target = int(total_samples * 0.25)
s_target = int(total_samples * 0.25)
o_target = int(total_samples * 0.25)

# Aplicar resampling (sobremuestreo o submuestreo)
f_resampled = resample(f_class, replace=True, n_samples=f_target, random_state=42)
b_resampled = resample(b_class, replace=True, n_samples=b_target, random_state=42)
s_resampled = resample(s_class, replace=True, n_samples=s_target, random_state=42)
o_resampled = resample(o_class, replace=True, n_samples=o_target, random_state=42)

# Combinar las clases resampleadas
df_resampled = pd.concat([f_resampled, b_resampled, s_resampled, o_resampled])

# Barajar el dataset para mezclar las clases
df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

df_exploded_filtered = df_resampled.copy() 

C:\Users\agusd\AppData\Local\Temp\ipykernel_27324\1115539204.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_class['y'] = 'o'


In [17]:
df_exploded_filtered[['Player', 'X', 'y']].head()

,Player,X,y
0,Daniil Medvedev,"[f29, f1, f1]",b
1,Andy Roddick,"[b38, f1, f3]",s
2,Marco Cecchinato,"[b38, b3, f1]",o
3,Rafael Nadal,"[4, f3, f3]",f
4,Novak Djokovic,"[f29, b2, b1]",b


In [18]:
df_exploded_filtered['y'].value_counts(normalize = True)

b    0.25
s    0.25
o    0.25
f    0.25
Name: y, dtype: float64

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

n_steps = 3

X = df_exploded_filtered['X']
players = df_exploded_filtered['Player']
y = df_exploded_filtered['y']

# Tokenizador para las secuencias
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(seq) for seq in X])
X_encoded = tokenizer.texts_to_sequences([' '.join(seq) for seq in X])
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_encoded, maxlen=n_steps)

# Codificar jugadores
player_encoder = LabelEncoder()
players_encoded = player_encoder.fit_transform(players)

# Codificar etiquetas (y)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [20]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

# Input 1: Secuencia
seq_input = Input(shape=(n_steps,), name='sequence_input')
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(seq_input)
lstm = LSTM(64)(embedding)

# Input 2: Jugador
player_input = Input(shape=(1,), name='player_input')
player_embedding = Embedding(input_dim=len(player_encoder.classes_), output_dim=5)(player_input)
player_flattened = tf.keras.layers.Flatten()(player_embedding)

# Combinar ambas entradas
combined = Concatenate()([lstm, player_flattened])

# Salida
output = Dense(y_categorical.shape[1], activation='softmax', name='output')(combined)

# Crear modelo
model = Model(inputs=[seq_input, player_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sequence_input      │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ player_input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 3, 50)     │      6,900 │ sequence_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 5)      │      3,975 │ player_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     29,440 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 5)         │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 69)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 4)         │        280 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 40,595 (158.57 KB)

 Trainable params: 40,595 (158.57 KB)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
# Dividir en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, players_train, players_test = train_test_split(
    X_padded, y_categorical, players_encoded, test_size=0.2, random_state=42
)

# Entrenar el modelo
history = model.fit(
    [X_train, players_train],
    y_train,
    validation_data=([X_test, players_test], y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['sequence_input', 'player_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


7438/7438 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.4118 - loss: 1.2586 - val_accuracy: 0.4372 - val_loss: 1.2189
Epoch 2/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.4416 - loss: 1.2173 - val_accuracy: 0.4433 - val_loss: 1.2154
Epoch 3/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4476 - loss: 1.2082 - val_accuracy: 0.4445 - val_loss: 1.2123
Epoch 4/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.4472 - loss: 1.2063 - val_accuracy: 0.4456 - val_loss: 1.2102
Epoch 5/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.4489 - loss: 1.2055 - val_accuracy: 0.4454 - val_loss: 1.2075
Epoch 6/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.4538 - loss: 1.1991 - val_accuracy: 0.4468 - val_loss: 1.2063
Epoch 7/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.4551 - loss: 1.1943 - val_accuracy: 0.4500 - val_loss: 1.2030
Epoch 8/10
7438/7438 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.4574 - loss: 1.1896 - val

In [22]:
# Predecir el próximo golpe
def predict_next_shot(sequence, player):
    # Preprocesar entrada
    seq_encoded = tokenizer.texts_to_sequences([sequence])
    seq_padded = tf.keras.preprocessing.sequence.pad_sequences(seq_encoded, maxlen=n_steps)
    player_encoded = player_encoder.transform([player])
    
    # Hacer predicción
    prediction = model.predict([seq_padded, player_encoded])[0]  # Vector de probabilidades para la fila
    
    # Mapear índices a las clases
    class_probabilities = {label_encoder.inverse_transform([i])[0]: prob for i, prob in enumerate(prediction)}
    return class_probabilities

# Evaluar en el conjunto de prueba
loss, accuracy = model.evaluate([X_test, players_test], y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

1860/1860 ━━━━━━━━━━━━━━━━━━━━ 2s 869us/step - accuracy: 0.4535 - loss: 1.1932
Test Loss: 1.1957441568374634, Test Accuracy: 0.4539022445678711


In [23]:
secuencia = ['f2', 's2', 'f2']

print(predict_next_shot(secuencia, "Roger Federer"))
print(predict_next_shot(secuencia, "Novak Djokovic"))
print(predict_next_shot(secuencia, "Rafael Nadal"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
{'b': 0.1681054, 'f': 0.114540145, 'o': 0.1543693, 's': 0.5629852}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


C:\Users\agusd\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['sequence_input', 'player_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


{'b': 0.3073151, 'f': 0.15559845, 'o': 0.2078012, 's': 0.32928523}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
{'b': 0.25600618, 'f': 0.14888479, 'o': 0.121123426, 's': 0.47398558}


A priori y sin ningun analisis mas extenso, el output es coherente. Federer (reves a una mano) es mas propenso al slice reves y el que tiene mas p de este golpe.
Djokovic, mas polivalente, tiene probabilidades similares.
Nadal mas defensivo tambien tiene mas chance de slice de reves.